Importing the dataset

In [1]:
import pandas as pd
lab1_df = pd.read_csv('https://data.cityofnewyork.us/api/views/5rq2-4hqu/rows.csv?accessType=DOWNLOAD')

In [2]:
lab1_df.head()

,created_at,tree_id,block_id,the_geom,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,...,st_assem,st_senate,nta,nta_name,boro_ct,state,Latitude,longitude,x_sp,y_sp
0,08/27/2015,180683,348711,POINT (-73.84421521958048 40.723091773924274),3,0,OnCurb,Alive,Fair,Acer rubrum,...,28,16,QN17,Forest Hills,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.768749
1,09/03/2015,200540,315986,POINT (-73.81867945834878 40.79411066708779),21,0,OnCurb,Alive,Fair,Quercus palustris,...,27,11,QN49,Whitestone,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.837379
2,09/05/2015,204026,218365,POINT (-73.93660770459083 40.717580740099116),3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,50,18,BK90,East Williamsburg,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.891267
3,09/05/2015,204337,217969,POINT (-73.93445615919741 40.713537494833226),10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,53,18,BK90,East Williamsburg,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.253136
4,08/30/2015,189565,223043,POINT (-73.97597938483258 40.66677775537875),21,0,OnCurb,Alive,Good,Tilia americana,...,44,21,BK37,Park Slope-Gowanus,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.425999


To start, create a Python list containing the name of the columns of interest. Then, create a new DataFrame called 'lab1_df_reduced' that only contains those columns.

In [3]:
name_list = ['spc_latin', 'spc_common', 'status', 'health', 'tree_dbh', 'stump_diam']
lab1_df_reduced = lab1_df[name_list]

In [4]:
lab1_df_reduced

,spc_latin,spc_common,status,health,tree_dbh,stump_diam
0,Acer rubrum,red maple,Alive,Fair,3,0
1,Quercus palustris,pin oak,Alive,Fair,21,0
2,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,3,0
3,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,10,0
4,Tilia americana,American linden,Alive,Good,21,0
...,...,...,...,...,...,...
683783,Quercus palustris,pin oak,Alive,Good,25,0
683784,Cladrastis kentukea,Kentucky yellowwood,Alive,Good,7,0
683785,Acer rubrum,red maple,Alive,Good,12,0
683786,Acer rubrum,red maple,Alive,Good,9,0


Next, we will want to scan our new DataFrame for NaN values that may hinder later analyses. To do so, run the following code:

In [5]:
nullValue_counts=lab1_df_reduced.isnull().sum()
nullValue_counts

spc_latin     31619
spc_common    31619
status            0
health        31616
tree_dbh          0
stump_diam        0
dtype: int64

Based on this, we can see that there are 31,619 NaN values in the DataFrame's '*health*' and name-related columns. Let's investigate futher by trying to figure out why that is.

We'll do so by selecting the rows that have NaNs from our reduced DataFrame with the next command.

In [6]:
NaNsOnly=lab1_df_reduced.loc[(lab1_df_reduced['spc_latin'].isnull() ) | (lab1_df_reduced['spc_common'].isnull() ) | (lab1_df_reduced['health'].isnull() ) ]
NaNsOnly.head()

,spc_latin,spc_common,status,health,tree_dbh,stump_diam
61,NaN,NaN,Dead,NaN,2,0
307,NaN,NaN,Stump,NaN,0,9
370,NaN,NaN,Dead,NaN,4,0
494,NaN,NaN,Stump,NaN,0,17
556,NaN,NaN,Dead,NaN,2,0


Let's run one more command so we can gather more evidence to access whether identification issues were the source of our NaNs.

In [7]:
pd.value_counts(NaNsOnly['status'])

status
Stump    17654
Dead     13961
Alive        6
Name: count, dtype: int64

Let's obtain the mean diameter for every tree, including those which were dead and/or unidentified.

When calculating the mean here, we should be warry of columns for which no diameter information was recorded as including them in our calculations will render them incorrect.

Running the following command, we can see that 278 of such rows exist.

In [8]:
lab1_df_reduced.loc[(lab1_df_reduced['tree_dbh'] == 0) & (lab1_df_reduced['stump_diam'] == 0)]

,spc_latin,spc_common,status,health,tree_dbh,stump_diam
644,NaN,NaN,Dead,NaN,0,0
3647,NaN,NaN,Dead,NaN,0,0
5291,Cotinus coggygria,smoketree,Alive,Good,0,0
6029,NaN,NaN,Dead,NaN,0,0
6114,NaN,NaN,Dead,NaN,0,0
...,...,...,...,...,...,...
677527,NaN,NaN,Dead,NaN,0,0
677676,NaN,NaN,Dead,NaN,0,0
677698,NaN,NaN,Dead,NaN,0,0
680500,Acer palmatum,Japanese maple,Alive,Good,0,0


As such, filter out these rows and save the new DataFrame under the name 'lab1_df_reduced_zerosFiltered'.

In [9]:
lab1_df_reduced_zerosFiltered = lab1_df_reduced[~((lab1_df_reduced['tree_dbh'] == 0) & (lab1_df_reduced['stump_diam'] == 0))]

In [10]:
lab1_df_reduced_zerosFiltered

,spc_latin,spc_common,status,health,tree_dbh,stump_diam
0,Acer rubrum,red maple,Alive,Fair,3,0
1,Quercus palustris,pin oak,Alive,Fair,21,0
2,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,3,0
3,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,10,0
4,Tilia americana,American linden,Alive,Good,21,0
...,...,...,...,...,...,...
683783,Quercus palustris,pin oak,Alive,Good,25,0
683784,Cladrastis kentukea,Kentucky yellowwood,Alive,Good,7,0
683785,Acer rubrum,red maple,Alive,Good,12,0
683786,Acer rubrum,red maple,Alive,Good,9,0


Next, peform the following steps:

1) Group the rows of the filtered DataFrame by the columns 'status', 'spc_latin', 'spc_common', and 'health'.

2) Add 'dropna=False' as an argument to groupby().

3) Use mean() to get the mean tree diameter for each group.

4) Save this as a DataFrame named 'lab1_df_reduced_zerosFiltered_grouped'

Hint: If you save the column names as a list, this will take two lines of code. If not, it will take up a single line.

In [11]:
lab1_df_reduced_zerosFiltered_grouped = lab1_df_reduced_zerosFiltered.groupby(['status', 'spc_latin', 'spc_common', 'health'], dropna=False).mean()
lab1_df_reduced_zerosFiltered_grouped

tree_dbh  \
status spc_latin                          spc_common    health              
Alive  Acer                               maple         Fair    11.903991   
                                                        Good    11.778865   
                                                        Poor    10.509434   
       Acer buergerianum                  trident maple Fair     9.750000   
                                                        Good     7.724138   
...                                                                   ...   
       NaN                                NaN           Good    13.000000   
                                                        Poor    11.000000   
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN     12.000000   
       NaN                                NaN           NaN      5.492017   
Stump  NaN                                NaN           NaN      0.000000   

                                                                stump_diam  
status spc_latin                          spc_common    health              
Alive  Acer                               maple         Fair      0.000000  
                                                        Good      0.000000  
                                                        Poor      0.000000  
       Acer buergerianum                  trident maple Fair      0.000000  
                                                        Good      0.000000  
...                                                                    ...  
       NaN                                NaN           Good      0.000000  
                                                        Poor      0.000000  
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN       0.000000  
       NaN                                NaN           NaN       0.000000  
Stump  NaN                                NaN           NaN      16.750481  

[398 rows x 2 columns]

While we've come quite far from our original DataFrame, we still want to modify this one. To get the DataFrame ready for that, reset its index columns.

In [12]:
lab1_df_reduced_zerosFiltered_grouped = lab1_df_reduced_zerosFiltered_grouped.reset_index()
lab1_df_reduced_zerosFiltered_grouped

,status,spc_latin,spc_common,health,tree_dbh,stump_diam
0,Alive,Acer,maple,Fair,11.903991,0.000000
1,Alive,Acer,maple,Good,11.778865,0.000000
2,Alive,Acer,maple,Poor,10.509434,0.000000
3,Alive,Acer buergerianum,trident maple,Fair,9.750000,0.000000
4,Alive,Acer buergerianum,trident maple,Good,7.724138,0.000000
...,...,...,...,...,...,...
393,Alive,NaN,NaN,Good,13.000000,0.000000
394,Alive,NaN,NaN,Poor,11.000000,0.000000
395,Dead,Gleditsia triacanthos var. inermis,honeylocust,NaN,12.000000,0.000000
396,Dead,NaN,NaN,NaN,5.492017,0.000000


Our next step will be to create a new column called 'diameter' that combines the 'tree_dbh' and 'stump_diam' columns. Since all rows have a non-zero value in one column but not both, it will be safe to sum the two columns.

In [13]:
lab1_df_reduced_zerosFiltered_grouped['diameter'] = lab1_df_reduced_zerosFiltered_grouped['tree_dbh'] + lab1_df_reduced_zerosFiltered_grouped['stump_diam']
lab1_df_reduced_zerosFiltered_grouped

,status,spc_latin,spc_common,health,tree_dbh,stump_diam,diameter
0,Alive,Acer,maple,Fair,11.903991,0.000000,11.903991
1,Alive,Acer,maple,Good,11.778865,0.000000,11.778865
2,Alive,Acer,maple,Poor,10.509434,0.000000,10.509434
3,Alive,Acer buergerianum,trident maple,Fair,9.750000,0.000000,9.750000
4,Alive,Acer buergerianum,trident maple,Good,7.724138,0.000000,7.724138
...,...,...,...,...,...,...,...
393,Alive,NaN,NaN,Good,13.000000,0.000000,13.000000
394,Alive,NaN,NaN,Poor,11.000000,0.000000,11.000000
395,Dead,Gleditsia triacanthos var. inermis,honeylocust,NaN,12.000000,0.000000,12.000000
396,Dead,NaN,NaN,NaN,5.492017,0.000000,5.492017


After this, run the following command to remove the 'tree_dbh' and 'stump_diam' columns:

In [14]:
lab1_df_reduced_zerosFiltered_grouped.drop(columns=['tree_dbh', 'stump_diam'], inplace=True)
lab1_df_reduced_zerosFiltered_grouped

,status,spc_latin,spc_common,health,diameter
0,Alive,Acer,maple,Fair,11.903991
1,Alive,Acer,maple,Good,11.778865
2,Alive,Acer,maple,Poor,10.509434
3,Alive,Acer buergerianum,trident maple,Fair,9.750000
4,Alive,Acer buergerianum,trident maple,Good,7.724138
...,...,...,...,...,...
393,Alive,NaN,NaN,Good,13.000000
394,Alive,NaN,NaN,Poor,11.000000
395,Dead,Gleditsia triacanthos var. inermis,honeylocust,NaN,12.000000
396,Dead,NaN,NaN,NaN,5.492017


Our final modification to this DataFrame will be to add counts for each row. To start, assign each row in the reduced but ungrouped DataFrame ('lab1_df_reduced') with a count of 1 using the code below:

In [15]:
lab1_df_reduced_counts=lab1_df_reduced.assign(count = 1)
lab1_df_reduced_counts.head()

,spc_latin,spc_common,status,health,tree_dbh,stump_diam,count
0,Acer rubrum,red maple,Alive,Fair,3,0,1
1,Quercus palustris,pin oak,Alive,Fair,21,0,1
2,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,3,0,1
3,Gleditsia triacanthos var. inermis,honeylocust,Alive,Good,10,0,1
4,Tilia americana,American linden,Alive,Good,21,0,1


Next, perform the following steps on the 'lab1_df_reduced_counts' DataFrame to produce the image below:

1) Group the rows of the filtered DataFrame by the columns 'status', 'spc_latin', 'spc_common', and 'health'.

2) Add 'dropna=False' as an argument to groupby().

3) Use sum() to add up the counts for each group.

4) Save this as a DataFrame named 'lab1_df_reduced_counts_grouped'

Hint: As before, this code will take up two lines if you use a list.

In [16]:
lab1_df_reduced_counts_grouped = lab1_df_reduced_counts.groupby(['status', 'spc_latin', 'spc_common', 'health'], dropna = False).sum()
lab1_df_reduced_counts_grouped

tree_dbh  \
status spc_latin                          spc_common    health             
Alive  Acer                               maple         Fair       20582   
                                                        Good       55396   
                                                        Poor        6684   
       Acer buergerianum                  trident maple Fair         195   
                                                        Good         672   
...                                                                  ...   
       NaN                                NaN           Good          52   
                                                        Poor          11   
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN           12   
       NaN                                NaN           NaN        76361   
Stump  NaN                                NaN           NaN            0   

                                                                stump_diam  \
status spc_latin                          spc_common    health               
Alive  Acer                               maple         Fair             0   
                                                        Good             0   
                                                        Poor             0   
       Acer buergerianum                  trident maple Fair             0   
                                                        Good             0   
...                                                                    ...   
       NaN                                NaN           Good             0   
                                                        Poor             0   
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN              0   
       NaN                                NaN           NaN              0   
Stump  NaN                                NaN           NaN         295713   

                                                                count  
status spc_latin                          spc_common    health         
Alive  Acer                               maple         Fair     1733  
                                                        Good     4708  
                                                        Poor      639  
       Acer buergerianum                  trident maple Fair       20  
                                                        Good       87  
...                                                               ...  
       NaN                                NaN           Good        4  
                                                        Poor        1  
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN         1  
       NaN                                NaN           NaN     13960  
Stump  NaN                                NaN           NaN     17654  

[398 rows x 3 columns]

With the 'count' column set up, the heavy-lifting has been done. To clean up, remove the unecessary columns and reset this DataFrame's index with this code here:

In [17]:
lab1_df_reduced_counts_grouped.drop(columns=['tree_dbh','stump_diam'], inplace=True)
lab1_df_reduced_counts_grouped=lab1_df_reduced_counts_grouped.reset_index()
lab1_df_reduced_counts_grouped.head()

,status,spc_latin,spc_common,health,count
0,Alive,Acer,maple,Fair,1733
1,Alive,Acer,maple,Good,4708
2,Alive,Acer,maple,Poor,639
3,Alive,Acer buergerianum,trident maple,Fair,20
4,Alive,Acer buergerianum,trident maple,Good,87


As the last modfication to our data, produce the final dataframe named 'lab1_df_final' by left-merging* (outer='left') the two DataFrames 'lab1_df_reduced_counts_grouped' and 'lab1_df_reduced_zerosFiltered_grouped', in that order from left to right.

*( Here we chose to left-merge because the right DataFrame might be missing some rows in a niche situation, removing trees from our dataset. Merging by union would also work. )

In [18]:
lab1_df_final = lab1_df_reduced_counts_grouped.merge(lab1_df_reduced_zerosFiltered_grouped, how='left')
lab1_df_final

,status,spc_latin,spc_common,health,count,diameter
0,Alive,Acer,maple,Fair,1733,11.903991
1,Alive,Acer,maple,Good,4708,11.778865
2,Alive,Acer,maple,Poor,639,10.509434
3,Alive,Acer buergerianum,trident maple,Fair,20,9.750000
4,Alive,Acer buergerianum,trident maple,Good,87,7.724138
...,...,...,...,...,...,...
393,Alive,NaN,NaN,Good,4,13.000000
394,Alive,NaN,NaN,Poor,1,11.000000
395,Dead,Gleditsia triacanthos var. inermis,honeylocust,NaN,1,12.000000
396,Dead,NaN,NaN,NaN,13960,5.492017


If you'd like, you can make it look a bit nicer in the notebook by running this:

In [19]:
lab1_df_final=lab1_df_final.set_index(['status','spc_latin','spc_common','health'])
lab1_df_final

count  \
status spc_latin                          spc_common    health          
Alive  Acer                               maple         Fair     1733   
                                                        Good     4708   
                                                        Poor      639   
       Acer buergerianum                  trident maple Fair       20   
                                                        Good       87   
...                                                               ...   
       NaN                                NaN           Good        4   
                                                        Poor        1   
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN         1   
       NaN                                NaN           NaN     13960   
Stump  NaN                                NaN           NaN     17654   

                                                                 diameter  
status spc_latin                          spc_common    health             
Alive  Acer                               maple         Fair    11.903991  
                                                        Good    11.778865  
                                                        Poor    10.509434  
       Acer buergerianum                  trident maple Fair     9.750000  
                                                        Good     7.724138  
...                                                                   ...  
       NaN                                NaN           Good    13.000000  
                                                        Poor    11.000000  
Dead   Gleditsia triacanthos var. inermis honeylocust   NaN     12.000000  
       NaN                                NaN           NaN      5.492017  
Stump  NaN                                NaN           NaN     16.750481  

[398 rows x 2 columns]

Finally, save the final DataFrame as a .csv file and submit it alongside your Google Colab notebook for grading.

In [20]:
lab1_df_final.to_csv('/content/lab1_final.csv')